In [1]:
print("Ok")

Ok


In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


c:\Users\Ayon\anaconda3\envs\llmapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%pwd

'd:\\Project\\Source-Code-Analysis-Project\\research'

In [4]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/langchain-ai/langchain.git", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/langchain-ai/langchain.git test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".md"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(page_content='# Global development guidelines for the LangChain monorepo\n\nThis document provides context to understand the LangChain Python project and assist with development.\n\n## Project architecture and context\n\n### Monorepo structure\n\nThis is a Python monorepo with multiple independently versioned packages that use `uv`.\n\n```txt\nlangchain/\n├── libs/\n│   ├── core/             # `langchain-core` primitives and base abstractions\n│   ├── langchain/        # `langchain-classic` (legacy, no new features)\n│   ├── langchain_v1/     # Actively maintained `langchain` package\n│   ├── partners/         # Third-party integrations\n│   │   ├── openai/       # OpenAI models and embeddings\n│   │   ├── anthropic/    # Anthropic (Claude) integration\n│   │   ├── ollama/       # Local model support\n│   │   └── ... (other integrations maintained by the LangChain team)\n│   ├── text-splitters/   # Document chunking utilities\n│   ├── standard-tests/   # Shared test suite for

*Context Aware Splitting*

In [9]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [10]:
texts = documents_splitter.split_documents(documents)


In [11]:
texts

[Document(page_content='# Global development guidelines for the LangChain monorepo\n\nThis document provides context to understand the LangChain Python project and assist with development.\n\n## Project architecture and context\n\n### Monorepo structure\n\nThis is a Python monorepo with multiple independently versioned packages that use `uv`.', metadata={'source': 'test_repo\\AGENTS.md', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='```txt\nlangchain/\n├── libs/\n│   ├── core/             # `langchain-core` primitives and base abstractions\n│   ├── langchain/        # `langchain-classic` (legacy, no new features)\n│   ├── langchain_v1/     # Actively maintained `langchain` package\n│   ├── partners/         # Third-party integrations\n│   │   ├── openai/       # OpenAI models and embeddings\n│   │   ├── anthropic/    # Anthropic (Claude) integration\n│   │   ├── ollama/       # Local model support', metadata={'source': 'test_repo\\AGENTS.md', 'language': <Language.

In [12]:
len(texts)

299

In [ ]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')



ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
embeddings=OpenAIEmbeddings(disallowed_special=())


In [ ]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')


Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://pla

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
vectordb.persist()


In [ ]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)


In [ ]:
question = "what is download_hugging_face_embeddings function doing?"


In [ ]:
result = qa(question)
print(result['answer'])

In [ ]:
question = "what is text_split function doing?"


In [ ]:
result = qa(question)
print(result['answer'])

In [ ]:
result

In [ ]:
question = "what is the use of download_hugging_face_embeddings?"


In [ ]:
result = qa(question)
print(result['answer'])